# DATA COLLECTION

This notebook focuses on the data collection. We will build our dataframe thanks to the spotify API.

### Import

In [4]:
# Librairies
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import time

## I - Using Spotify's API

Nous avons d'abord essayé de créer une base de données grâce à l'API Spotify.

### I. 1. Construction d'un premier dataset

Tout d'abord, Spotify ne met pas à disposition une quelconque base de donnée déjà constituée, pas même une liste de morceaux. Nous allons donc devoir faire des requêtes successives à l'API pour récolter les informations sur les morceaux choisis, afin de constituer notre premier dataset. Chaque requête permet d'obtenir les metadonnées d'un artiste, d'une playlist ou d'un morceau donné. C'est ce que nous avons choisi de faire.

Pour minimiser le nombre de requêtes, nous sélectionnons une playlist de 10 000 morceaux déjà faite par un utilisateur Spotify, afin d'obtenir des informations sur chaque morceau qui la compose : titre, id, artist, artist_id. Ensuite, nous récupérons les informations qui nous intéressent sur chaque morceau : les track features, qui sont un tas d'indices quantitatifs Enfin, nous récupérons le genre qui n'est associé qu'à l'artiste, et pas au morceau. C'est ici une limite préoccupante : le genre qui est la variable à prédire n'est en fait que le genre de l'artiste et pas le genre véritable du morceau.

D'abord, on initialise le client Spotify pour l'obtention des données via l'API grâce au token d'accès.

In [13]:
def spotify_client():
    """
    Initialize the Spotify API client with client credentials.
    Returns an authenticated Spotify client.
    """
    return spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id="d666ee3ae4c94b85945c3dba39776f4f",
        client_secret="c1973a77acbe48c0b2f105e4f57d7d46" 
    ))

Ensuite, nous utilisons ce client pour notre requête sur la playlist de 10 000 morceaux.

Tout d'abord, nous définissions la fonction `fetch_playlist_tracks`, ci-dessous, qui permet l'obtention d'informations identifiant l'ensemble des musiques d'une playlist.

In [ ]:
def fetch_playlist_tracks(playlist_id):
    """
    Fetch all tracks from a Spotify playlist. 
    Params:
        -playlist_id: Spotify playlist ID.
    Returns a list of dictionaries containing track details.
    """
    tracks = []
    results = spotify_client().playlist_tracks(playlist_id)
    while results:
        for item in results['items']:
            track = item['track']
            if track:  # Ensure the track is not None
                tracks.append(track)
        results = spotify_client().next(results) if results['next'] else None
    return tracks

Puis nous voulons pour chacun de ces morceaux des informations quantitatives. Pour cela nous utilisons `fetch_track_data` qui requête l'api pour récupérer ces données.

Il semblerait que l'API impose une limitation, en effet, lorsqu'on demande trop d'informations à la fois, une erreur est raised. Voilà pourquoi nous décidons de récolter les informations de 100 morceaux à la fois pour éviter tout risque d'erreur. 

In [17]:
def fetch_track_data(tracks):
    """
    Fetch metadata and audio features for each track in the playlist.
    Params:
        -tracks: List of tracks from the playlist.
    Returns a list of dictionaries containing track metadata and audio features.
    """
    track_data = []
    i=0
    j=0
    for track in tracks:
        track_id = track['id']
        i+=1
        print(i)
        audio_features = spotify_client().audio_features([track_id])[0]
        genre=fetch_artist_genre(track)
        if audio_features:  # Ensure audio features are available
            artist_name = ", ".join([artist['name'] for artist in track['artists']])
            dict_track={"track Name": track['name'],
                "artists": artist_name,
                "track_id": track_id,
                "popularity": track['popularity'],
                "duration_ms": track['duration_ms'],
                "explicit": track['explicit'], 'genre': genre}
            for key in audio_features.keys():
                dict_track[key]=audio_features[key]
            track_data.append(dict_track)
        if i==100:
            j+=1
            save_to_csv(track_data, f"intermédiaire{j}")
            i=0
            time.sleep(60)
    return track_data

Enfin, nous voulons accéder au genre de chaque morceau. L'API associe le genre d'un morceau à son artiste. En outre, l'API n'associe pas un unique genre à un morceau, mais une liste de genres. C'est une autre limite préoccupante : comment choisir le genre de la musique parmi plusieurs genres correspondant à l'artiste ? Nous proposons d'abord d'associer à la musique le genre de l'artiste, mais d'autres solutions auraient pu être envisageables.

In [18]:
def fetch_artist_genre(track):
    """This function fetches the genre of an artist with a track from this artist, it will be 
    considerated as the genre of the song later

    Args:
        track a dict the countains infos about the track

    Returns: genre a string that is the genre of an artist
        
    """
    artist=track['artists'][0]['id']
    if spotify_client().artist(artist)['genres'] != []:
        return spotify_client().artist(artist)['genres'][0]
    else:
        return 'N/A'

Enfin, pour pouvoir manipuler facilement nos données sans avoir à reconstituer notre base de données à chaque fois qu'on veut l'utiliser, nous décidons d'enregistrer le df ainsi obtenu grâce à la fonction `save_to_csv` ci-dessous :

In [19]:
def save_to_csv(data, filename):
    """
    Save the list of track data to a CSV file.
    Params:
    :param data: List of dictionaries containing track details.
    :param filename: Output CSV file name.
    """
    keys = data[0].keys() if data else []
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"Data saved to {filename}")


On peut maintenant, générer une première base de données en combinant les différentes fonctions définies ci-dessus.

In [ ]:
playlist_ids = []

track_data=[]

for playlist in playlist_ids: 
    tracks = fetch_playlist_tracks(playlist)
    
    track_data+=(fetch_track_data(tracks))

save_to_csv(track_data, "datafram.csv")

### I. 2. Modification des règlementations autour de l'utilisation de l'API spotify

https://developer.spotify.com/blog/2024-11-27-changes-to-the-web-api

Spotify introduced some changes to the Web API that we were using. Since November 27, 2024, new Web API no longer allows us to access or use the following endpoints and functionality such as Audio Features, Audio Analysis, and Get Featured Playlists. We can not access to the metadata of tracks or playlists on Spotify anymore.

This means that the code we used to query the API to create `intermediaire3.csv` is no longer functional. We can no longer generate dataframes from the Spotify API.

We tried using the APIs of other platforms such as Deezer, Last.FM, or MusicBrainz. Unfortunately, none of these APIs provide information that could help identify the genre of a song. While the Spotify API offers all the audio features of tracks (such as key, danceability, liveness, etc.), the APIs of its competitors do not provide access to this data at all; most only allow retrieving a track's ID.

## II - Using a Kaggle DataFrame

Since we can no longer use the Spotify API to build our code, the only dataset we generated is `intermediaire3.csv`. We created this dataset as a test, so it’s quite small (only 1500 rows). Since we were able to collect data using the Spotify API but can no longer collect more, we decided to use another dataset.

We will use a dataframe created in the same way as our `intermediaire3.csv`, which is available on Kaggle. Indeed, this dataframe was built using the Spotify API and contains the same features, such as danceability, energy, or key, for instance. We are making this dataset change solely to improve the performance of our model.

We decided to store this database on S3 (Datalab storage) so that it is easily accessible to us.

In [ ]:
import pandas as pd

file_path = "/tlaflotte/genre_detector/spotify_tracks.csv"

df = pd.read_csv("https://minio.lab.sspcloud.fr" + file_path)

Now that we have imported our complete database, we can have a quick visualization to ensure the features are the same.

In [1]:
df.head()

NameError: name 'df' is not defined